# Xata | Xata

> [Xata](https://xata.io)は、PostgreSQLをベースにしたサーバーレスデータプラットフォームです。データベースとの対話にはPython SDKを提供し、データの管理にはUIを提供しています。Xataにはネイティブのベクタータイプがあり、任意のテーブルに追加可能で、類似性検索に対応しています。LangChainはベクターを直接Xataに挿入し、与えられたベクターの最も近い隣接点をクエリすることで、Xataを使ってLangChainの埋め込み統合機能を全て利用できます。
>
> > [Xata](https://xata.io) is a serverless data platform, based on PostgreSQL. It provides a Python SDK for interacting with your database, and a UI for managing your data.
> > Xata has a native vector type, which can be added to any table, and supports similarity search. LangChain inserts vectors directly to Xata, and queries it for the nearest neighbors of a given vector, so that you can use all the LangChain Embeddings integrations with Xata.




このノートブックは、XataをVectorStoreとして使用する方法を案内します。

> This notebook guides you how to use Xata as a VectorStore.




## Setup | セットアップ

### Create a database to use as a vector store | ベクトルストアとして使用するデータベースを作成する

[Xata UI](https://app.xata.io)で新しいデータベースを作成します。名前は何でも構いませんが、このノートパッドでは`langchain`という名前を使用します。次に、テーブルを作成します。これも名前は何でも良いですが、ここでは`vectors`という名前を使います。UIを通じて以下のカラムを追加してください：

> In the [Xata UI](https://app.xata.io) create a new database. You can name it whatever you want, in this notepad we'll use `langchain`.
> Create a table, again you can name it anything, but we will use `vectors`. Add the following columns via the UI:

* `content`は「テキスト」タイプです。これは`Document.pageContent`の値を格納するために使用されます。

  > `content` of type "Text". This is used to store the `Document.pageContent` values.

* `embedding`は「Vector」型を使用します。使用するモデルに合わせた次元を選んでください。このノートブックでは、1536次元のOpenAIの埋め込みを使用しています。

  > `embedding` of type "Vector". Use the dimension used by the model you plan to use. In this notebook we use OpenAI embeddings, which have 1536 dimensions.

* `search`は"Text"型です。この例ではメタデータのカラムとして使用されます。

  > `search` of type "Text". This is used as a metadata column by this example.

* メタデータとして使用したい他の列です。これらは `Document.metadata` オブジェクトから取得されます。例えば、`Document.metadata` オブジェクトに `title` プロパティがある場合、テーブルに `title` 列を作成すれば、それが自動的に埋められます。

  > any other columns you want to use as metadata. They are populated from the `Document.metadata` object. For example, if in the `Document.metadata` object you have a `title` property, you can create a `title` column in the table and it will be populated.





まず最初に、依存関係をインストールしましょう：

> Let's first install our dependencies:




In [ ]:
!pip install xata openai tiktoken langchain

OpenAIのキーを環境に読み込みましょう。もし持っていない場合は、OpenAIのアカウントを作成し、この[ページ](https://platform.openai.com/account/api-keys)でキーを作成することができます。

> Let's load the OpenAI key to the environemnt. If you don't have one you can create an OpenAI account and create a key on this [page](https://platform.openai.com/account/api-keys).




In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

同様に、Xataの環境変数を取得する必要があります。新しいAPIキーを作成するには、[アカウント設定](https://app.xata.io/settings)にアクセスしてください。データベースURLを見つけるためには、作成したデータベースの設定ページに移動します。データベースURLはおおよそ次のような形になります：`https://demo-uni3q8.eu-west-1.xata.sh/db/langchain`。

> Similarly, we need to get the environment variables for Xata. You can create a new API key by visiting your [account settings](https://app.xata.io/settings). To find the database URL, go to the Settings page of the database that you have created. The database URL should look something like this: `https://demo-uni3q8.eu-west-1.xata.sh/db/langchain`.




In [ ]:
api_key = getpass.getpass("Xata API key: ")
db_url = input("Xata database URL (copy it from your DB settings):")

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.xata import XataVectorStore

### Create the Xata vector store | Xataベクトルストアを作成する

テストデータセットをインポートしましょう：

> Let's import our test dataset:




In [ ]:
loader = TextLoader("../../modules/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

次に、Xataテーブルをバックエンドにした実際のベクトルストアを作成します。

> Now create the actual vector store, backed by the Xata table.




In [ ]:
vector_store = XataVectorStore.from_documents(
    docs, embeddings, api_key=api_key, db_url=db_url, table_name="vectors"
)

上記のコマンドを実行した後、Xata UIにアクセスすると、埋め込み情報とともにドキュメントがロードされていることが確認できます。既にベクター内容を含むXataテーブルを使用する場合は、XataVectorStoreコンストラクタを初期化する必要があります。

> After running the above command, if you go to the Xata UI, you should see the documents loaded together with their embeddings.
> To use an existing Xata table that already contains vector contents, initialize the XataVectorStore constructor:




In [ ]:
vector_store = XataVectorStore(
    api_key=api_key, db_url=db_url, embedding=embeddings, table_name="vectors"
)

### Similarity Search | 類似性検索




In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
found_docs = vector_store.similarity_search(query)
print(found_docs)

### Similarity Search with score (vector distance) | スコア（ベクトル距離）を用いた類似性検索




In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
result = vector_store.similarity_search_with_score(query)
for doc, score in result:
    print(f"document={doc}, score={score}")